In [1]:
# Parth korat
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\parth\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [3]:
# Visit the URL with San Diego County zip codes
url = 'https://www.zillow.com/browse/homes/ca/san-diego-county/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# find 'ul' tag with the class 'bh-body-links' tag that contains all zip codes
ul =soup.find('ul', class_='bh-body-links')

sd_zip_codes = []

# find the 'li' tag and get all children under the tag
zip_code_tags = ul.findChildren('li', recursive=False)

# run a for loop to get all zip codes, then append to zip_codes list
for zip_code_tag in zip_code_tags:
    sd_zip_codes.append(zip_code_tag.text.strip('\n'))

# show first 5 zip codes from the scrape
sd_zip_codes[:5]

['91910', '92101', '92126', '91911', '92592']

In [6]:
# create dictionary with the 'San Diego County Zip Codes' as the key and the zip_codes list as the values
dict_zip_codes = {'San Diego County Zip Codes': sd_zip_codes}

# read the dictionary as a DataFrame object
zip_codes_df = pd.DataFrame(dict_zip_codes)

len(zip_codes_df)

192

In [7]:
url_2 = 'https://www.greatschools.org/california/schools/?gradeLevels%5B%5D=e&gradeLevels%5B%5D=m&gradeLevels%5B%5D=h&gradeLevels%5B%5D=p&tableView=Overview&view=table'
browser.visit(url_2)

In [8]:
html_2 = browser.html
soup_2 = bs(html_2, 'html.parser')

In [9]:
# find the 'table' tag that includes all schools
table = soup_2.find('table')

# get the 'tbody' tag that is a child of the table tag
tbody = table.findChild('tbody')

# within the tbody tag there are multiple 'tr' tags that contain one school each on that page
school_tags = tbody.findChildren('tr', recursive=False)
all_zip_codes =[]

# loop through the list and append the zip code from address to a list
for school_tag in school_tags:

    # core includes the name, rating, and address of the school
    core = school_tag.findChildren('td')[0]

    # find name in the core
    name = core.find('a', class_='name').text
    # find rating in the core
    rating = core.find('div', class_='circle-rating--small circle-rating--10').text.split('/')[0]
    # find address in the core
    address = core.find('div', class_='address').text

    # find rest of the data in the table by indexing
    district_type = school_tag.findChildren('td')[1].text
    grades = school_tag.findChildren('td')[2].text
    num_students = school_tag.findChildren('td')[3].text
    stu_teach_ratio = school_tag.findChildren('td')[4].text
    district = school_tag.findChildren('td')[6].text

    # strip the address by the comma and get the last item in the list to get the zip
    current_zip = address.split(',')[-1].strip(' ')
    
    if current_zip in sd_zip_codes:
        print(name, rating, current_zip, address, district_type, grades, num_students, stu_teach_ratio, district)



Canyon Crest Academy 10 92130 5951 Village Center Loop Road, San Diego, CA, 92130 Public district 9-12 2572 27:1 San Dieguito Union High
Great Oak High 10 92592 32555 Deer Hollow Way, Temecula, CA, 92592 Public district 9-12 3161 30:1 Temecula Valley Unified
Del Norte High 10 92127 16601 Nighthawk Lane, San Diego, CA, 92127 Public district 9-12 2351 26:1 Poway Unified
